<a href="https://colab.research.google.com/github/choi4490/AAA/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%B5%9C%EC%9B%90%EC%9A%B0(2025).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 아파트건설 예측(클로드도움)

In [ ]:
아파트 건설공사 데이터를 활용한 머신러닝 모델링을 위해 분석해드리겠습니다.
최적 모델링 기법 분석

1. 추천 모델링 기법
주요 추천 모델:
Random Forest: 건설 프로젝트의 복잡한 비선형 관계를 잘 포착하며, 특성 중요도 분석 가능
XGBoost/LightGBM: 높은 예측 정확도와 결측값 처리 능력
Multiple Linear Regression: 해석 가능성이 높아 건설업계에서 선호
Neural Network: 복잡한 패턴 학습 가능하지만 데이터 수가 제한적
50개 데이터 고려시 최적 선택:
Random Forest (1순위): 소규모 데이터셋에서도 안정적 성능
XGBoost (2순위): 과적합 방지 기능 우수
Ensemble 방법: 여러 모델 조합으로 안정성 향상

2. 데이터 전처리 고려사항
물가상승률 정규화: 기준년도 대비 조정
공사유형 인코딩: 원핫인코딩 또는 라벨인코딩
시계열 특성: 시작년월을 기준으로 한 경과시간 변환
이상치 처리: 건설비용의 극값 처리

모델 활용 가이드

3. 데이터 준비사항
필수 전처리:
결측값 처리 (평균값 대체 또는 도메인 지식 활용)
이상치 탐지 및 처리 (IQR 방법 사용)
물가지수 정규화 (기준년도 설정)
범주형 변수 인코딩

4. 모델 성능 향상 방안
데이터 부족 해결책:
데이터 증강: 유사 프로젝트 데이터 수집
외부 데이터 활용: 건설물가정보, 지역별 인건비 등
도메인 지식 활용: 건설업계 전문가 피드백 반영
모델 앙상블:

5. 실무 적용 시 고려사항
모델 업데이트:
신규 프로젝트 완료시 데이터 추가 학습
계절성 및 경제 상황 변화 반영
정기적 모델 재훈련 (분기별 권장)
예측 신뢰도 관리:
예측 구간(Prediction Interval) 제공
유사 과거 프로젝트 참조 정보 제공
전문가 검토 프로세스 포함
주요 예측 타겟별 특징:
공사기간: 날씨, 인허가 지연 등 외부 요인 고려
비용: 원자재 가격 변동성 높음, 월별 업데이트 필요
인력: 숙련도별 차등 적용, 지역별 인력 수급 상황 반영
50개 데이터로는 Deep Learning보다 Random Forest + XGBoost 앙상블이 최적이며, 지속적인 데이터 수집과 모델 업데이트가 예측 정확도 향상의 핵심입니다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class ConstructionCostPredictor:
    def __init__(self):
        self.models = {}
        self.scalers = {}
        self.encoders = {}
        self.feature_importance = {}

    def load_and_preprocess_data(self, data_path=None):
        """
        데이터 로드 및 전처리
        실제 사용시 data_path에 Excel/CSV 파일 경로 입력
        """
        # 샘플 데이터 생성 (실제 데이터로 교체 필요)
        np.random.seed(42)
        n_samples = 50

        data = {
            # 시간 관련
            '시작년월': pd.date_range('2020-01', periods=n_samples, freq='M'),
            '준공년월': pd.date_range('2021-01', periods=n_samples, freq='M'),
            '전체예산실행승인년월': pd.date_range('2019-12', periods=n_samples, freq='M'),

            # 물가 관련
            '공사기간내물가상승률': np.random.uniform(0.02, 0.08, n_samples),
            '철근레미콘물가상승률': np.random.uniform(0.03, 0.12, n_samples),
            '보통인부단가': np.random.uniform(150000, 200000, n_samples),

            # 프로젝트 특성
            '공사유형': np.random.choice(['아파트', '오피스텔', '주상복합'], n_samples),
            '지하층수': np.random.randint(1, 4, n_samples),
            '지상최고층수': np.random.randint(10, 35, n_samples),
            '아파트동수': np.random.randint(3, 15, n_samples),
            '세대수': np.random.randint(200, 1500, n_samples),
            '연면적': np.random.uniform(30000, 150000, n_samples),
            '대지면적': np.random.uniform(10000, 50000, n_samples),
            '마감수준': np.random.choice(['A', 'B', 'C', 'D'], n_samples),

            # 인력 관련
            '현장직원수': np.random.randint(15, 50, n_samples),

            # 비용 관련
            '외주계약금액': np.random.uniform(20000000000, 80000000000, n_samples),
            '원재료금액': np.random.uniform(15000000000, 60000000000, n_samples),
            '경비금액': np.random.uniform(2000000000, 8000000000, n_samples),
            '도급금액': np.random.uniform(50000000000, 200000000000, n_samples),
        }

        df = pd.DataFrame(data)

        # 타겟 변수 생성 (실제 데이터에서는 실제 값 사용)
        df['최종공사개월수'] = (df['준공년월'] - df['시작년월']).dt.days / 30
        df['실행예산금액'] = df['도급금액'] * np.random.uniform(0.85, 1.15, n_samples)
        df['실제투입원가'] = df['실행예산금액'] * np.random.uniform(0.9, 1.1, n_samples)
        df['적정HR인사발령'] = np.round(df['연면적'] / 3000 + df['세대수'] / 100).astype(int)

        return df

    def feature_engineering(self, df):
        """특성 엔지니어링"""
        df_processed = df.copy()

        # 날짜 특성 추출
        df_processed['시작년도'] = df_processed['시작년월'].dt.year
        df_processed['시작월'] = df_processed['시작년월'].dt.month
        df_processed['준공년도'] = df_processed['준공년월'].dt.year

        # 프로젝트 규모 지표
        df_processed['세대당연면적'] = df_processed['연면적'] / df_processed['세대수']
        df_processed['동당세대수'] = df_processed['세대수'] / df_processed['아파트동수']
        df_processed['용적률'] = df_processed['연면적'] / df_processed['대지면적']

        # 비용 효율성 지표
        df_processed['세대당도급금액'] = df_processed['도급금액'] / df_processed['세대수']
        df_processed['연면적당도급금액'] = df_processed['도급금액'] / df_processed['연면적']

        # 물가 조정된 비용 (2020년 기준)
        base_year = 2020
        df_processed['물가조정계수'] = (1 + df_processed['공사기간내물가상승률']) ** (df_processed['시작년도'] - base_year)
        df_processed['물가조정도급금액'] = df_processed['도급금액'] / df_processed['물가조정계수']

        # 카테고리 인코딩
        le_type = LabelEncoder()
        le_finish = LabelEncoder()

        df_processed['공사유형_encoded'] = le_type.fit_transform(df_processed['공사유형'])
        df_processed['마감수준_encoded'] = le_finish.fit_transform(df_processed['마감수준'])

        self.encoders['공사유형'] = le_type
        self.encoders['마감수준'] = le_finish

        return df_processed

    def prepare_features(self, df):
        """모델링용 특성 준비"""
        feature_columns = [
            '지하층수', '지상최고층수', '아파트동수', '세대수', '연면적', '대지면적',
            '현장직원수', '공사기간내물가상승률', '철근레미콘물가상승률', '보통인부단가',
            '외주계약금액', '원재료금액', '경비금액', '도급금액', '물가조정도급금액',
            '공사유형_encoded', '마감수준_encoded', '시작년도', '시작월',
            '세대당연면적', '동당세대수', '용적률', '세대당도급금액', '연면적당도급금액'
        ]

        target_columns = ['최종공사개월수', '실행예산금액', '실제투입원가', '적정HR인사발령']

        X = df[feature_columns]
        y = df[target_columns]

        return X, y, feature_columns, target_columns

    def train_models(self, X, y, target_columns):
        """다중 타겟 모델 훈련"""
        # 특성 스케일링
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        self.scalers['features'] = scaler

        # 각 타겟별로 모델 훈련
        for target in target_columns:
            print(f"\n=== {target} 모델 훈련 중 ===")

            # 타겟 스케일링
            target_scaler = StandardScaler()
            y_target = target_scaler.fit_transform(y[target].values.reshape(-1, 1)).ravel()
            self.scalers[target] = target_scaler

            # 데이터 분할
            X_train, X_test, y_train, y_test = train_test_split(
                X_scaled, y_target, test_size=0.2, random_state=42
            )

            # 모델들 정의
            models = {
                'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
                'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42),
                'LinearRegression': LinearRegression()
            }

            best_model = None
            best_score = -np.inf

            # 각 모델 훈련 및 평가
            for name, model in models.items():
                # 교차 검증
                cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
                mean_cv_score = np.mean(cv_scores)

                # 전체 데이터로 훈련
                model.fit(X_train, y_train)

                # 테스트 예측
                y_pred = model.predict(X_test)
                test_score = r2_score(y_test, y_pred)

                print(f"{name}: CV R² = {mean_cv_score:.3f}, Test R² = {test_score:.3f}")

                if mean_cv_score > best_score:
                    best_score = mean_cv_score
                    best_model = model

            self.models[target] = best_model

            # 특성 중요도 저장 (Random Forest의 경우)
            if hasattr(best_model, 'feature_importances_'):
                self.feature_importance[target] = best_model.feature_importances_

    def predict(self, X_new):
        """새로운 데이터에 대한 예측"""
        # 특성 스케일링
        X_scaled = self.scalers['features'].transform(X_new)

        predictions = {}
        for target, model in self.models.items():
            # 예측
            y_pred_scaled = model.predict(X_scaled)

            # 역스케일링
            y_pred = self.scalers[target].inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
            predictions[target] = y_pred

        return predictions

    def plot_feature_importance(self, feature_columns, target='최종공사개월수'):
        """특성 중요도 시각화"""
        if target in self.feature_importance:
            importance = self.feature_importance[target]
            feature_imp_df = pd.DataFrame({
                'feature': feature_columns,
                'importance': importance
            }).sort_values('importance', ascending=False)

            plt.figure(figsize=(10, 8))
            sns.barplot(data=feature_imp_df.head(15), x='importance', y='feature')
            plt.title(f'{target} - 특성 중요도')
            plt.xlabel('중요도')
            plt.tight_layout()
            plt.show()

            return feature_imp_df

    def evaluate_models(self, X_test, y_test, target_columns):
        """모델 성능 평가"""
        results = {}

        for target in target_columns:
            if target in self.models:
                model = self.models[target]

                # 예측
                X_scaled = self.scalers['features'].transform(X_test)
                y_pred_scaled = model.predict(X_scaled)

                # 역스케일링
                y_pred = self.scalers[target].inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
                y_true = y_test[target].values

                # 평가 지표 계산
                mae = mean_absolute_error(y_true, y_pred)
                mse = mean_squared_error(y_true, y_pred)
                rmse = np.sqrt(mse)
                r2 = r2_score(y_true, y_pred)

                results[target] = {
                    'MAE': mae,
                    'RMSE': rmse,
                    'R²': r2
                }

                print(f"\n{target} 성능:")
                print(f"  MAE: {mae:,.2f}")
                print(f"  RMSE: {rmse:,.2f}")
                print(f"  R²: {r2:.3f}")

        return results

# 사용 예제
def main():
    # 모델 인스턴스 생성
    predictor = ConstructionCostPredictor()

    # 데이터 로드 및 전처리
    print("데이터 로드 및 전처리 중...")
    df = predictor.load_and_preprocess_data()
    df_processed = predictor.feature_engineering(df)

    # 특성 및 타겟 준비
    X, y, feature_columns, target_columns = predictor.prepare_features(df_processed)

    print(f"특성 수: {len(feature_columns)}")
    print(f"샘플 수: {len(X)}")
    print(f"타겟 변수: {target_columns}")

    # 모델 훈련
    print("\n모델 훈련 시작...")
    predictor.train_models(X, y, target_columns)

    # 신규 프로젝트 예측 예제
    print("\n=== 신규 프로젝트 예측 예제 ===")

    # 신규 프로젝트 데이터 (예시)
    new_project = pd.DataFrame({
        '지하층수': [2],
        '지상최고층수': [25],
        '아파트동수': [8],
        '세대수': [800],
        '연면적': [80000],
        '대지면적': [25000],
        '현장직원수': [30],
        '공사기간내물가상승률': [0.05],
        '철근레미콘물가상승률': [0.08],
        '보통인부단가': [170000],
        '외주계약금액': [40000000000],
        '원재료금액': [30000000000],
        '경비금액': [5000000000],
        '도급금액': [100000000000],
        '물가조정도급금액': [95000000000],
        '공사유형_encoded': [0],  # 실제로는 인코더 사용
        '마감수준_encoded': [1],  # 실제로는 인코더 사용
        '시작년도': [2024],
        '시작월': [3],
        '세대당연면적': [100],
        '동당세대수': [100],
        '용적률': [3.2],
        '세대당도급금액': [125000000],
        '연면적당도급금액': [1250000]
    })

    # 예측 수행
    predictions = predictor.predict(new_project)

    print("\n신규 프로젝트 예측 결과:")
    for target, pred_value in predictions.items():
        if target == '최종공사개월수':
            print(f"  {target}: {pred_value[0]:.1f} 개월")
        elif target == '적정HR인사발령':
            print(f"  {target}: {int(pred_value[0])} 명")
        else:
            print(f"  {target}: {pred_value[0]:,.0f} 원")

if __name__ == "__main__":
    main()

# 실제 사용을 위한 추가 함수들

def load_excel_data(file_path):
    """Excel 파일에서 데이터 로드"""
    try:
        df = pd.read_excel(file_path)
        print(f"데이터 로드 완료: {len(df)}개 레코드")
        return df
    except Exception as e:
        print(f"데이터 로드 실패: {e}")
        return None

def save_model(predictor, model_path):
    """모델 저장"""
    import pickle
    with open(model_path, 'wb') as f:
        pickle.dump(predictor, f)
    print(f"모델 저장 완료: {model_path}")

def load_model(model_path):
    """모델 로드"""
    import pickle
    with open(model_path, 'rb') as f:
        predictor = pickle.load(f)
    print(f"모델 로드 완료: {model_path}")
    return predictor

# 데이터 품질 확인 함수
def check_data_quality(df):
    """데이터 품질 확인"""
    print("=== 데이터 품질 확인 ===")
    print(f"전체 레코드 수: {len(df)}")
    print(f"전체 컬럼 수: {len(df.columns)}")
    print(f"결측값 비율:")
    print((df.isnull().sum() / len(df) * 100).round(2))
    print(f"\n수치형 변수 기본 통계:")
    print(df.describe())